# Предсказание цены автомобиля


In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

import warnings 
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('C:/Users/Win 10/Downloads/train_car.csv')

In [3]:
train.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Ignis 1.2 AMT Delta BSIV,2019,525000,20000,Petrol,Individual,Automatic,First Owner,20.89 kmpl,1197 CC,81.80 bhp,113Nm@ 4200rpm,5.0
1,Tata Indigo CS LX (TDI) BS-III,2015,250000,80000,Diesel,Individual,Manual,Second Owner,19.09 kmpl,1405 CC,69.01 bhp,135Nm@ 2500rpm,5.0
2,Renault KWID RXT Optional,2016,250000,22000,Petrol,Individual,Manual,First Owner,25.17 kmpl,799 CC,53.3 bhp,72Nm@ 4386rpm,5.0
3,Land Rover Discovery Sport TD4 SE,2019,4500000,21000,Diesel,Dealer,Automatic,First Owner,12.83 kmpl,1999 CC,148.31 bhp,382nm@ 1750-2250rpm,7.0
4,Hyundai Verna 1.6 VTVT S Option,2015,700000,25000,Petrol,Individual,Manual,First Owner,17.01 kmpl,1591 CC,121.4 bhp,154.9Nm@ 4200rpm,5.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6502 entries, 0 to 6501
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6502 non-null   object 
 1   year           6502 non-null   int64  
 2   selling_price  6502 non-null   int64  
 3   km_driven      6502 non-null   int64  
 4   fuel           6502 non-null   object 
 5   seller_type    6502 non-null   object 
 6   transmission   6502 non-null   object 
 7   owner          6502 non-null   object 
 8   mileage        6328 non-null   object 
 9   engine         6328 non-null   object 
 10  max_power      6332 non-null   object 
 11  torque         6327 non-null   object 
 12  seats          6328 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 660.5+ KB


In [5]:
train = train.dropna()

In [6]:
y = train.selling_price
train = train.drop('selling_price',1)

# Преобразования

In [7]:
def convert_categ(df):
    df['name'] = df['name'].apply(lambda x : x.split()[0])
    
    df['mileage'] = df['mileage'].str.replace(r'[^\d.]', '', regex=True).apply(lambda x : float(x))

    df['engine'] = df['engine'].str.replace(r'[^\d.]', '', regex=True).apply(lambda x : float(x))

    df['max_power'] = df['max_power'].str.replace(r'[^\d.]', '', regex=True).apply(lambda x : float(x))
    
    df['torque'] = df['torque'].str.replace(r'[^\d.]', '_', regex=True).apply(lambda x: float(str(x).split('_')[0]))

In [8]:
convert_categ(train)
train

,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti,2019,20000,Petrol,Individual,Automatic,First Owner,20.89,1197.0,81.80,113.0,5.0
1,Tata,2015,80000,Diesel,Individual,Manual,Second Owner,19.09,1405.0,69.01,135.0,5.0
2,Renault,2016,22000,Petrol,Individual,Manual,First Owner,25.17,799.0,53.30,72.0,5.0
3,Land,2019,21000,Diesel,Dealer,Automatic,First Owner,12.83,1999.0,148.31,382.0,7.0
4,Hyundai,2015,25000,Petrol,Individual,Manual,First Owner,17.01,1591.0,121.40,154.9,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6497,Skoda,2016,11000,Petrol,Dealer,Automatic,First Owner,14.30,1598.0,103.50,153.0,5.0
6498,Maruti,2012,60000,Petrol,Individual,Manual,First Owner,19.00,998.0,66.10,90.0,5.0
6499,Maruti,2019,5621,Petrol,Trustmark Dealer,Automatic,First Owner,22.00,1197.0,81.80,113.0,5.0
6500,Hyundai,2012,120000,Petrol,Individual,Manual,Second Owner,21.10,814.0,55.20,74.5,5.0


In [9]:
categor_index = train.select_dtypes(include=['object']).columns
numerical_index = train.select_dtypes(include=[np.number]).columns
steps = [('Categorical', OneHotEncoder(handle_unknown='ignore'), categor_index), ('Numerical', MinMaxScaler(), numerical_index)]
preprocessor = ColumnTransformer(steps)
# label encoder показывал результаты хуже

In [10]:
train = preprocessor.fit_transform(train)

# Модель

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.1, random_state=42)

In [12]:
#model = RandomForestRegressor()
#paramgrid = {  'max_depth': [10,20,30,50],
#               'n_estimators': [100,300,400,500,1000]}
#RFRgrid = GridSearchCV(model, param_grid=paramgrid,scoring='neg_mean_squared_error',cv=10)
#RFRgrid_result = RFRgrid.fit(X_train, y_train)

In [13]:
best_model = RandomForestRegressor(n_estimators = 300, max_depth = 20 , random_state = 42)
best_model.fit(X_train,y_train)
#линейные модели сильно проигрывают в качестве

RandomForestRegressor(max_depth=20, n_estimators=300, random_state=42)

In [14]:
MAE(y_test,best_model.predict(X_test))

64498.412665858275

# Тест

In [15]:
test = pd.read_csv('C:/Users/Win 10/Downloads/test_car.csv')

In [16]:
convert_categ(test)

In [17]:
test.fillna(test.median(), inplace = True)

In [18]:
test = preprocessor.transform(test)

In [19]:
best_model.predict(test)

array([284784.48323192, 326187.1713034 , 199035.4176936 , ...,
       749930.64309873, 376146.32207323, 398725.76451259])